# Microclimate sensors data


<span style="font-size: 15px;"> The following assessment explores the 'microclimate-sensors-data' dataset from the City of Melbourne's open data platform. The assessment will cover basic exploration and transformation to prepare the data for visualisation in Power BI. You can access the source data as well as information about the dataset from the below link.  </span>

https://data.melbourne.vic.gov.au/explore/dataset/microclimate-sensors-data/information/


<span style="font-size: 15px;"> The dataset contains climate readings from a handful sensors located within Melbourne. The data is updated every fifteen minutes and includes information about wind-speed, direction, temperature, humidity, pollutants, and atmospheric pressure. It is described by the City of Melbourne as being useful to determine variations in microclimate changes throughout the day. </span>



<span style="font-size: 15px;"> The assessment is not required to be completed 'live', i.e. you can work on it at your pace and in your own time. When completing the assessment, **you are welcome and encouraged to use ChatGPT, Google, StackOverflow, and any other tool which supports you** since everyone uses these tools on the job. With that in mind, please do ensure that you are able to explain the approach you've taken. </span>


<img src="https://www.hobodataloggers.com.au/images/thumbs/0011187_hobo-advanced-weather-station-kit.png" />


*** 

<div class="alert alert-block alert-info">
<b>Note:</b> The first cell of this notebook downloads the data required and loads it to session memory as a dataframe. You will not be assessed on this, however please ensure you run this cell to load the file. Alternatively you're welcome to delete this cell if you'd prefer to load it separately however do not modify the data.
</div>


In [ ]:
import requests
import pandas as pd
import os
url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/microclimate-sensors-data/exports/csv?lang=en&timezone=Australia%2FMelbourne&use_labels=true&delimiter=%2C'
response = requests.get(url)
if response.status_code==200:
    file_path = 'microclimate_sensors_data.csv'
    with open (file_path, 'wb') as file:
        file.write(response.content)

df = pd.read_csv(file_path)
print("The file has downloaded to:", os.getcwd(),"... with filename:", file_path)

### 1. Print the column names of the dataframe

In [ ]:
df.columns

### 2. Show the 'shape' of the dataframe
<div class="alert alert-block alert-info">
<b>Note:</b> In this context 'shape' simply refers to the row/column length of the dataframe</div>

In [ ]:
df.shape

### 3. Produce summary statistics about the dataset

In [ ]:
df.describe()

### 4. Check whether there are blanks in the LatLong column. If there are blanks, how many are there?

In [ ]:
len(df.loc[df['LatLong'].isna()])

### 5. If there are blanks in the LatLong column, populate the blank values with the appropriate coordinates.
<div class="alert alert-block alert-info">
<b>Note:</b> Each sensor will contains at least one valid LatLong coordinate despite other records being missing, you can use these coordinates to populate blanks for a given sensor. There should only be one unique LatLong coordinate for each sensor.</div>

In [ ]:
df['LatLong'] = df.groupby('Device_id')['LatLong'].apply(lambda x: x.ffill().bfill())

In [ ]:
# Are there any blank LatLongs?
df[['Device_id','LatLong']].loc[df['LatLong'].isna()]

In [ ]:
# C
for device in set(df['Device_id']):
    print(device)
    print("Number of unique coordinates:", len(df.loc[df['Device_id']==f'{device}']['LatLong'].unique()))
    print("List of coordinates:", df.loc[df['Device_id']==f'{device}']['LatLong'].unique())
    print("-----------")

### 6. How many unique Device_id's are in the dataset?


In [ ]:
len(df['Device_id'].unique())

### 7. How many unique Device_id's contain one or more blank AirTemperature records?


In [ ]:
len(
    df['Device_id'].loc[df['AirTemperature'].isna()].unique()
    )

### 8. What is the largest value of MaximumWindDirection?

In [ ]:
max(df['MaximumWindDirection'])

### 9. Is MinimumWindDirection ever greater than MaximumWindDirection?

In [ ]:
df.loc[(df['MinimumWindDirection'] > df['MaximumWindDirection'])]

### 10. Split LatLong into two separate columns named 'Latitude' and 'Longitude'

In [ ]:
latlong_split = df['LatLong'].str.split(',',expand=True)
latlong_split.columns = ['Latitude','Longitude']

df = pd.concat([df, latlong_split], axis=1)
df.drop(columns=['LatLong'],inplace=True)
df.columns

### 11. Drop MinimumWindDirection, MaximumWindDirection, and AverageWindDirection from the dataset

In [ ]:
df.drop(columns=['MinimumWindDirection', 'MaximumWindDirection', 'AverageWindDirection'],inplace=True)
df.columns

### 12. Save the dataframe as a .csv to your computer

In [ ]:
df.to_csv('Python_transformed_dataset.csv')